In [ ]:
import pandas as pd
import requests
from tempfile import mkdtemp
from zipfile import ZipFile
import os
import json
import shutil

In [ ]:
SCANR_DUMP_URL = 'https://scanr-data.s3.gra.io.cloud.ovh.net/production/organizations.jsonl.gz'

In [3]:
def get_last_ror_dump_url():
    try:
        ROR_URL = 'https://zenodo.org/api/records/?communities=ror-data&sort=mostrecent'
        response = requests.get(url=ROR_URL).json()
        ror_dump_url = response['hits']['hits'][0]['files'][-1]['links']['self']
        print(f'Last ROR dump url found: {ror_dump_url}')
    except:
        ror_dump_url = 'https://zenodo.org/api/files/25d4f93f-6854-4dd4-9954-173197e7fad7/v1.1-2022-06-16-ror-data.zip'
        print(f'ROR dump url detection failed, using {ror_dump_url} instead')
    return ror_dump_url
ROR_DUMP_URL = get_last_ror_dump_url()
CHUNK_SIZE = 128
def download_ror_data() -> list:
    print('download ROR')
    ror_downloaded_file = 'ror_data_dump.zip'
    ror_unzipped_folder = mkdtemp()
    response = requests.get(url=ROR_DUMP_URL, stream=True)
    with open(file=ror_downloaded_file, mode='wb') as file:
        for chunk in response.iter_content(chunk_size=CHUNK_SIZE):
            file.write(chunk)
    with ZipFile(file=ror_downloaded_file, mode='r') as file:
        file.extractall(ror_unzipped_folder)
    for data_file in os.listdir(ror_unzipped_folder):
        if data_file.endswith('.json'):
            with open(f'{ror_unzipped_folder}/{data_file}', 'r') as file:
                data = json.load(file)
    os.remove(path=ror_downloaded_file)
    shutil.rmtree(path=ror_unzipped_folder)
    return data

ror_data = download_ror_data()

grid_to_ror = {}
for r in ror_data:
    grid = (r['external_ids'].get('GRID', {}).get('all'))
    if grid is None:
        continue
    if len(grid) in [11, 12, 13]:
        grid_to_ror[grid] = r['id'].split('/')[-1]
    else:
        print(grid)


Last ROR dump url found: https://zenodo.org/api/files/d72c6d64-be22-4040-ac02-6ce5451aaca1/v1.23-2023-04-12-ror-data.zip
download ROR


In [4]:
df_paysage = pd.read_json('paysage.jsonl', lines=True, dtype = {'siren': str, 'siret':str, 'idref':str, 'ed':str})
#df_paysage['name_normalized'] = df_paysage['name'].apply(lambda x:normalize(x))

In [5]:
df_scanr = pd.read_json(SCANR_DUMP_URL, lines=True)

scanr_data = df_scanr.to_dict(orient='records')
df_scanr['scanr_id'] = df_scanr['id']
df_scanr = df_scanr.set_index('id')

rnsr_actif = []
for e in scanr_data:
    to_keep = False
    if e['status'] != 'active':
        continue
    current_rnsr = None
    for idf in e.get('externalIds', []):
        if idf.get('type') == 'rnsr':
            to_keep = True
            current_rnsr = idf['id']
            e['rnsr'] = current_rnsr
    if to_keep:
        rnsr_actif.append(e)
len(rnsr_actif)

4610

In [6]:
def enrich_with_paysage(elt):
    enrich = None
    
    #elt['name_normalized'] = normalize(elt['scanr_label'])
    
    for f in ['grid', 'ror', 'siret', 'siren', 'idref']:
        if elt.get(f):
            current_id = elt[f]
            df_current = df_paysage[df_paysage[f] == current_id]
            if len(df_current) == 1:
                #print(f)
                enrich = df_current.to_dict(orient='records')[0]
                break
            if len(df_current) > 1:
                df_current2 = df_current[df_current.city == elt['scanr_city']]
                if len(df_current2) == 1:
                    enrich = df_current2.to_dict(orient='records')[0]
                    break
    if enrich:
        elt['paysage_label'] = enrich['name']
        elt['paysage_city'] = enrich['city']
        elt['paysage_country'] = enrich['country']
        for f in ['rnsr', 'paysage', 'siret', 'siren', 'wikidata', 'idref', 'ror', 'grid', 'ed']:
            if enrich.get(f) and enrich[f] != 'None':
                if f in elt and elt[f] != enrich[f]:
                    #print(f, elt[f], enrich[f])
                    pass
                elt[f] = enrich[f]
    else:
        pass
        #print('----')
        #print('not found in paysage')
        #print("'"+elt['scanr_id']+"',")
        #print('-----')
    for f in elt.copy():
        if elt[f] == None or elt[f] == 'None':
            del elt[f]
    return elt

In [7]:
def get_main(x):
    for e in x:
        if e.get('main') == True:
            return e
    return None

In [8]:
def get_city(e):
    add = get_main(e['address'])
    if add:
        return add.get('city', '')
    return ''

def get_country(e):
    add = get_main(e['address'])
    if add:
        return add.get('country', '')
    return ''

In [9]:
df_tutelles = pd.read_csv('tutelles.csv')
rnsr_tutelles = {}
for row in df_tutelles.itertuples():
    linked_rnsr = row.linked_rnsr.split('|')
    for r in linked_rnsr:
        if r not in rnsr_tutelles:
            rnsr_tutelles[r] = []
        tmp = row._asdict()
        elt = {}
        for f in ['ror', 'idref', 'paysage_label', 'paysage', 'wikidata']:
            if isinstance(tmp.get(f), str):
                elt[f] = tmp[f]
        rnsr_tutelles[r].append(elt)
        

In [10]:
rnsr_tutelles['202324384J']

[{'ror': '037tw3m02',
  'paysage_label': 'MBS',
  'paysage': '5HerM',
  'wikidata': 'Q3117571'}]

In [11]:
labos = []
for jx, e in enumerate(rnsr_actif):
    if jx%100 == 0:
        print(jx, end=',')
    elt = {'scanr_id': e['id'], 'scanr_label': e['label'].get('default'), 
           'scanr_city': get_city(e), 'scanr_country': get_country(e), 
           'scanr_acronym': e.get('acronym', {}).get('default')}
    
    elt['tutelles'] = []
    current_rnsr = e['rnsr']
    if current_rnsr in rnsr_tutelles:
        elt['tutelles'] = rnsr_tutelles[current_rnsr]
        
    elt['nb_tutelles'] = len(elt['tutelles'])
    
    elt['tutelles_paysage'] = '|'.join([k.get('paysage') for k in elt['tutelles'] if 'paysage' in k])
            
    for g in e.get('externalIds', []):
        if g['type'] in ['rnsr', 'idref', 'ror', 'grid', 'label_numero']:
            value = g['id'].replace('idref', '')
            if g['type'] not in elt:
                elt[g['type']] = value
    if elt.get('grid') and not elt.get('ror'):
        if elt['grid'] in grid_to_ror:
            elt['ror'] = grid_to_ror[elt['grid']]
    elt = enrich_with_paysage(elt)
    labos.append(elt)

0,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,2100,2200,2300,2400,2500,2600,2700,2800,2900,3000,3100,3200,3300,3400,3500,3600,3700,3800,3900,4000,4100,4200,4300,4400,4500,4600,

In [12]:
df_labos = pd.DataFrame(labos)

In [13]:
elt

{'scanr_id': '202224401G',
 'scanr_label': 'Laboratoire Interdisciplinaire Territoire Histoire Organisation RégulAtion Loi',
 'scanr_city': 'La Rochelle',
 'scanr_country': 'France',
 'scanr_acronym': 'LITHORAL',
 'tutelles': [{'ror': '04mv1z119',
   'idref': '035375043',
   'paysage_label': 'La Rochelle Université',
   'paysage': 'atbEK',
   'wikidata': 'Q1500822'}],
 'nb_tutelles': 1,
 'tutelles_paysage': 'atbEK',
 'rnsr': '202224401G'}

In [14]:
df_labos.scanr_country.value_counts(dropna=False)

France            4374
                   110
NaN                 92
Singapour            4
Japon                4
Canada               4
Etats-Unis           4
Chili                3
Inde                 2
Mexique              2
Corée du Sud         1
Sénégal              1
Egypte               1
Autriche             1
Brésil               1
Chine                1
Argentine            1
Russie               1
Ethiopie             1
Israël               1
Afrique du Sud       1
Name: scanr_country, dtype: int64

In [15]:
def match_ror(elt):

    params = {'type': "ror", 'year': "2022", 
                            'name': elt.get('scanr_label'),
                            'city': elt.get('scanr_city'),
                            'country': elt.get('scanr_country')
                           }
    if elt.get('scanr_acronym'):
        params['acronym'] = elt.get('scanr_acronym')
    supervisors_name = [k.get('paysage_label') for k in elt.get('tutelles', []) if k.get('paysage_label')]
    if supervisors_name:
        params['supervisor_name'] = ' '.join(supervisors_name)
    r = requests.post('https://affiliation-matcher.staging.dataesr.ovh/match', 
                      json=params)
    return r.json()['results']
                            

In [16]:
match_ror(labos[2])

['00t9y0z13']

In [17]:
for ix, elt in enumerate(labos):
    
    if 'match_ror' in elt:
        del elt['match_ror']
    
    if ix%50 == 0:
        print(ix, end=',')
    
    rors = match_ror(elt)
    if len(rors) == 1:
        elt['match_ror'] = rors[0]
        if 'ror' in elt:
            if elt['ror'] != rors[0]:
                print('------')
                print("MISMATCH ! " + rors[0])
                print(elt)
                print('------')
    elif len(rors) == 0:
        continue
    else:
        print('-----------------')
        print('multiple rors !')
        print(elt)
        print(rors)
        print('-----------------')
    

0,50,100,150,200,250,300,350,400,450,500,550,600,650,700,750,800,850,900,950,1000,1050,-----------------
multiple rors !
{'scanr_id': '201119517W', 'scanr_label': "Maison des sciences de l'homme de Montpellier", 'scanr_city': 'Montpellier', 'scanr_country': 'France', 'scanr_acronym': 'MSH M', 'tutelles': [{'ror': '05q3vnk25', 'idref': '050165224', 'paysage_label': 'Institut de recherche pour le développement', 'paysage': 'Uan45', 'wikidata': 'Q2910510'}, {'ror': '051escj72', 'paysage_label': 'Université de Montpellier', 'paysage': 'evv7S'}, {'ror': '00qhdy563', 'idref': '026404702', 'paysage_label': 'Université Paul-Valéry - Montpellier 3', 'paysage': '1I7hJ', 'wikidata': 'Q2912244'}, {'ror': '05kpkpg04', 'idref': '026643790', 'paysage_label': 'Centre de coopération internationale en recherche agronomique pour le développement', 'paysage': '7dd3j', 'wikidata': 'Q2945274'}], 'nb_tutelles': 4, 'tutelles_paysage': 'Uan45|evv7S|1I7hJ|7dd3j', 'rnsr': '201119517W', 'label_numero': 'FED 4156'

2250,2300,2350,------
MISMATCH ! 035xkbk20
{'scanr_id': '201220324U', 'scanr_label': 'Aix-Marseille school of economics', 'scanr_city': 'Marseille', 'scanr_country': 'France', 'scanr_acronym': 'AMSE', 'tutelles': [{'ror': '02feahw73', 'idref': '02636817X', 'paysage_label': 'Centre national de la recherche scientifique', 'paysage': 'n2X5f', 'wikidata': 'Q280413'}, {'ror': '02d9dg697', 'idref': '026374889', 'paysage_label': 'École des hautes études en sciences sociales', 'paysage': 'y52D7', 'wikidata': 'Q273518'}, {'ror': '040baw385', 'idref': '163078998', 'paysage_label': 'Centrale Méditerranée', 'paysage': '1tI7C', 'wikidata': 'Q273454'}, {'ror': '035xkbk20', 'idref': '15863621X', 'paysage_label': 'Aix-Marseille Université', 'paysage': 'xJdyB', 'wikidata': 'Q2302586'}], 'nb_tutelles': 4, 'tutelles_paysage': 'n2X5f|y52D7|1tI7C|xJdyB', 'rnsr': '201220324U', 'grid': 'grid.462942.f', 'idref': '034789731', 'label_numero': 'UMR 7316', 'ror': '02ek9wp67', 'match_ror': '035xkbk20'}
------
2400

In [18]:
# doublons ['04xz2my07', '0473hxz56'] pour 201622179T

In [19]:
df_labos = pd.DataFrame(labos)

for ix, row in df_labos.iterrows():
    if isinstance(row.ror, str):
        df_labos.at[ix, 'new_ror'] = row.ror
    elif 'match_ror' in row and isinstance(row.match_ror, str):
        df_labos.at[ix, 'new_ror'] = row.match_ror

In [20]:
df_labos_final = df_labos[['rnsr', 'new_ror', 'scanr_label', 'scanr_acronym', 'scanr_city', 'tutelles_paysage', 'nb_tutelles', 'idref', 'paysage', 'label_numero']]
df_labos_final.columns = ['rnsr', 'ror', 'scanr_label', 'scanr_acronym', 'scanr_city', 'tutelles_paysage', 'nb_tutelles', 'idref', 'paysage', 'label_numero']

In [21]:
df_labos_final.to_csv('labos.csv', index=False)

In [22]:
df_note = pd.DataFrame([{'note': 'Le RNSR 201622179T apparaît en doublon dans ROR : 04xz2my07 et 0473hxz56'}])

In [23]:
df_ed = pd.read_csv('ed.csv')

In [24]:
def get_stat(df):
    stats = {}
    total = len(df)
    stats['nb_total'] = total
    for f in ['ror', 'paysage', 'idref']:
        nb = len(df[f].dropna())
        stats[f'nb_{f}'] = nb
        stats[f'percentage_{f}'] = nb/total
    return stats

In [25]:
stats_tutelles = get_stat(df_tutelles)
stats_labos = get_stat(df_labos_final)
stats_ed = get_stat(df_ed)
df_stats = pd.DataFrame({'tutelles':stats_tutelles, 'laboratoires': stats_labos, 'ED': stats_ed}).reset_index()
df_stats.columns=['indicator', 'tutelles', 'laboratoires', 'ED']

In [26]:
writer = pd.ExcelWriter("rnsr_ror.xlsx", engine="xlsxwriter")
df_stats.to_excel(writer, sheet_name="bilan", index=False)
df_tutelles.to_excel(writer, sheet_name="tutelles", index=False)
df_labos_final.to_excel(writer, sheet_name="labos", index=False)
df_ed.to_excel(writer, sheet_name="ED", index=False)
df_note.to_excel(writer, sheet_name="notes", index=False)
writer.close()

In [27]:
df_stats

,indicator,tutelles,laboratoires,ED
0,nb_total,385.000000,4610.000000,274.000000
1,nb_ror,342.000000,1067.000000,0.000000
2,percentage_ror,0.888312,0.231453,0.000000
3,nb_paysage,384.000000,575.000000,274.000000
4,percentage_paysage,0.997403,0.124729,1.000000
5,nb_idref,276.000000,1964.000000,271.000000
6,percentage_idref,0.716883,0.426030,0.989051
